In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [5]:
df = pd.read_csv('./data/vectors_and_result.csv')

In [6]:
df.head(5)

,Unnamed: 0,sbert_encoded,word2vec_encoded,encoded_result
0,0,"[-0.15123848617076874, 0.08008230477571487, -0...",[ 9.95774269e-02 2.39087641e-02 4.54255566e-...,8
1,1,"[0.028186652809381485, 0.11452296376228333, -0...",[ 1.77159026e-01 -2.92782895e-02 -1.58776551e-...,0
2,2,"[-0.03124922700226307, 0.07508113980293274, -0...",[ 7.65224844e-02 2.61926323e-01 -2.87270933e-...,24
3,3,"[-0.01341341994702816, 0.044576939195394516, 0...",[ 6.15313798e-02 3.28309268e-01 -4.76197958e-...,4
4,4,"[-0.09738583862781525, 0.02743050456047058, 0....",[-7.48965964e-02 1.06664538e-01 -1.65855251e-...,23


In [7]:
from sklearn.model_selection import train_test_split

In [14]:
def parse_vector(vec_str):
    vec_str = vec_str.strip('[]').replace(',', ' ')
    return np.array([float(x) for x in vec_str.split() if x], dtype=float)


In [15]:
df['sbert_encoded'] = df['sbert_encoded'].progress_apply(parse_vector)

100%|██████████| 73343/73343 [00:27<00:00, 2705.61it/s]


In [16]:
# df_sampled = (
#     df.groupby("result_Encoded")
#       .sample(n=4000, random_state=42)
#       .reset_index(drop=True)
# )

# X = np.vstack(df_sampled["vectors"].values)
# Y = df_sampled["result_Encoded"].values
X = np.vstack(df['sbert_encoded'].values)
Y = df['encoded_result'].values

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
df['encoded_result'].value_counts()

encoded_result
8     3720
3     3720
5     3717
11    3655
13    3509
7     3132
15    3131
17    3129
10    3128
21    3060
14    3059
24    3058
23    3056
20    3056
16    3027
22    3002
12    2969
25    2158
18    2158
9     2154
0     2150
2     2040
4     2038
6     1736
19    1283
26    1265
1     1233
Name: count, dtype: int64

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [12]:
# param_grid = {
#     'C': [0.1, 1, 10],
#     'gamma': ['scale', 'auto', 0.1, 0.01],
#     'kernel': ['rbf', 'poly'],
# }
# grid = GridSearchCV(SVC(), param_grid, cv=3, scoring='f1_weighted', n_jobs=-1)
# grid.fit(X_train, Y_train)

# print(grid.best_params_)

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# pipeline = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf', LogisticRegression(
#         multi_class='multinomial',
#         solver='lbfgs',
#         C=3.0,
#         max_iter=2000,
#         n_jobs=-1
#     ))
# ])

# pipeline.fit(X, Y)

c:\Users\ASUS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('clf',
                 LogisticRegression(C=3.0, max_iter=2000,
                                    multi_class='multinomial', n_jobs=-1))])

In [24]:
# model = LogisticRegression(
#     multi_class='multinomial',     # use softmax, NOT one-vs-rest
#     solver='lbfgs',                # best for multinomial + dense data
#     max_iter=2000,                 # ensure convergence
#     C=3.0,                         # lower regularization = better boundary fit
#     class_weight=None,             # your data is balanced (4k/class)
#     n_jobs=-1  
# )

model = MLPClassifier(
    hidden_layer_sizes=(512, 256, 128),
    activation='relu',
    solver='adam',
    learning_rate_init=0.0003,
    alpha=0.0005,                # L2 regularization (avoid overfitting)
    batch_size=64,               # smaller → better generalization
    max_iter=10000,
    early_stopping=False,
    n_iter_no_change=50,
    validation_fraction=0.15,
    beta_1=0.9,
    beta_2=0.999,
    verbose=True
)

# model = RandomForestClassifier(
#     n_estimators=200,
#     max_depth=30,
#     min_samples_split=5,
#     min_samples_leaf=2,
#     class_weight='balanced',
#     n_jobs=-1,
#     random_state=42,
#     verbose=True
# )

In [25]:
model.fit(X_train,Y_train)
# model.fit(X_train_scaled,Y_train)

Iteration 1, loss = 0.26938005
Iteration 2, loss = 0.04695631
Iteration 3, loss = 0.02557065
Iteration 4, loss = 0.01998688
Iteration 5, loss = 0.01606189
Iteration 6, loss = 0.01465461
Iteration 7, loss = 0.01446303
Iteration 8, loss = 0.01238049
Iteration 9, loss = 0.01278942
Iteration 10, loss = 0.01125256
Iteration 11, loss = 0.01156691
Iteration 12, loss = 0.01101922
Iteration 13, loss = 0.01313345
Iteration 14, loss = 0.00861384
Iteration 15, loss = 0.01042340
Iteration 16, loss = 0.01135052
Iteration 17, loss = 0.00975134
Iteration 18, loss = 0.00918198
Iteration 19, loss = 0.01183999
Iteration 20, loss = 0.00874881
Iteration 21, loss = 0.01004140
Iteration 22, loss = 0.01117455
Iteration 23, loss = 0.01012180
Iteration 24, loss = 0.00899874
Iteration 25, loss = 0.01114165
Iteration 26, loss = 0.00875562


c:\Users\ASUS\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(alpha=0.0005, batch_size=64, hidden_layer_sizes=(512, 256, 128),
              learning_rate_init=0.0003, max_iter=10000, n_iter_no_change=50,
              validation_fraction=0.15, verbose=True)

In [ ]:
# from sklearn.calibration import CalibratedClassifierCV
# calibrated_svc = CalibratedClassifierCV(model, cv=3)
# calibrated_svc.fit(X_train_scaled, Y_train)
# probs = calibrated_svc.predict_proba(X_test)    

c:\Users\ASUS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\ASUS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\ASUS\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [34]:
import pickle    

In [35]:
with open ("./pickle_files/classification_model.pkl","wb") as f:
    pickle.dump(model,f)

In [27]:
# preds = model.predict(X_test)
preds = model.predict(X_test)

In [28]:
train_preds = model.predict(X_train)

In [ ]:
# caliberated_preds = calibrated_svc.predict(X_test_scaled)

In [29]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [30]:
precision = precision_score(Y_test,preds, average='weighted')
recall = recall_score(Y_test,preds, average='weighted')
f1 = f1_score(Y_test,preds, average='weighted')

In [31]:
print(f"precision : {precision}")
print(f"recall : {recall}")
print(f"f1 : {f1}")

precision : 0.9867293966798171
recall : 0.9867066603040425
f1 : 0.9866867760812998


In [32]:
train_precision = precision_score(Y_train,train_preds, average='weighted')
train_recall = recall_score(Y_train,train_preds, average='weighted')
train_f1 = f1_score(Y_train,train_preds, average='weighted')

In [33]:
print(f"Train precision : {train_precision}")
print(f"Train recall : {train_recall}")
print(f"Train f1 : {train_f1}")

Train precision : 0.9993540168828348
Train recall : 0.9993523536830623
Train f1 : 0.9993524845531699


In [ ]:
# caliberated_precision = precision_score(Y_test,caliberated_preds,average='weighted')
# caliberated_recall = recall_score(Y_test,caliberated_preds,average='weighted')
# caliberated_f1 = f1_score(Y_test,caliberated_preds,average='weighted')

In [ ]:
# print(f"caliberated_precision : {caliberated_precision}")
# print(f"caliberated_recall : {caliberated_recall}")
# print(f"caliberated_f1 : {caliberated_f1}")



caliberated_precision : 0.40686297765678847
caliberated_recall : 0.41984375
caliberated_f1 : 0.4069682503977317
